In [5]:
using HDF5
p::Array{Float64} = Float64[]
t::Array{Float64} = Float64[]
u::Array{Float64} = Float64[]
v::Array{Float64} = Float64[]
x::Array{Float64} = Float64[]
y::Array{Float64} = Float64[]

h5open("../sims/navier_stokes/ns_result.hdf5", "r") do file
    global p = read(file, "result")["value"]["p"]["value"]
    global t = read(file, "result")["value"]["t"]["value"]
    global u = read(file, "result")["value"]["u"]["value"]
    global v = read(file, "result")["value"]["v"]["value"]
    global x = read(file, "result")["value"]["x"]["value"]
    global y = read(file, "result")["value"]["y"]["value"]
end

h5open("../sims/navier_stokes/ns_result.hdf5", "r")

🗂️ HDF5.File: (read-only) ../sims/navier_stokes/ns_result.hdf5
└─ 📂 result
   ├─ 🏷️ OCTAVE_NEW_FORMAT
   ├─ 🔢 type
   └─ 📂 value
      ├─ 📂 p
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 t
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 u
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 v
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 x
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      └─ 📂 y
         ├─ 🏷️ OCTAVE_NEW_FORMAT
         ├─ 🔢 type
         └─ 🔢 value

In [6]:
include("../functions/GreedyRegression.jl")
include("../functions/Integrate.jl")
include("../functions/ReportModel.jl")
include("../functions/EnvelopePol.jl")
include("../functions/PickSubdomains.jl")

using .GreedyRegression, .Integrate, .ReportModel, .EnvelopePol, .PickSubdomains

nl::Int = 2 # under-estimate this
nw::Int = 128 # number of domains we integrate over 
dof::Int = 1 # scalars have one degree of freedom
dim::Int = 3 # how many dimensions does our data have?
env_pow::Int = 4 # weight is (1-x^2)^power
size_vec::Vector{Int} = [64,64,16] # how many gridpoints should we use per integration?
buffer::Int = 0 # Don't use points this close to boundary

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)

size_of_data::Vector{Int} = [size(u,1), size(u,2), size(u,3)]
seed::Int = 10
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

grid::Vector{Vector{Float64}} = [vec(y), vec(x), vec(t)]
corners

3×128 Matrix{Int64}:
 20   6  11  12  30  35  51  39  45  …  36  60  26  30   9  19  31  37   1
 26  50  45  24  29   2  48  30  19     42  10  64  62  24  29  12  37  52
 26  54  42  70  74   9  51  65  44     27  48  77  41  58   8  77  67  68

In [7]:
using Statistics

G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString, data::AbstractArray, derivs::AbstractVector{Int64}, scale::AbstractFloat)
    col::Vector{Float64} = Integrate.integrate(data, derivs, grid, corners, size_vec, pol)

    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, scale)
end

add_library_term("∂u/∂x", u, [2], 1.)
add_library_term("∂v/∂x", v, [2], 1.)
add_library_term("∂u/∂y", u, [1], 1.)
add_library_term("∂v/∂y", v, [1], 1.)
add_library_term("∂u/∂t", u, [3], 1.)
add_library_term("∂v/∂t", v, [3], 1.)
add_library_term("∂uu/∂x", u .* u, [2], 1.)
add_library_term("∂uv/∂x", u .* v, [2], 1.)
add_library_term("∂uv/∂y", u .* v, [1], 1.)
add_library_term("∂vv/∂y", v .* v, [1], 1.)
add_library_term("∂^2u/∂x^2", u, [2,2], 1.)
add_library_term("∂^2u/∂y^2", u, [1,1], 1.)
add_library_term("∂^2v/∂x^2", v, [2,2], 1.)
add_library_term("∂^2v/∂y^2", v, [1,1], 1.)
add_library_term("∂P/∂x", p, [2], 1.)
add_library_term("∂P/∂y", p, [1], 1.)

norm_vec::Vector{Float64} = Integrate.integrate(0*u .+ 1, Int64[], grid, corners, size_vec, pol)
G ./= norm_vec
G ./= scales'

G

128×16 Matrix{Float64}:
  0.312239    0.0304484   -0.0281448   -0.312239   …   0.0608608    0.066376
  0.0797725  -0.217798     0.299316    -0.0797725     -0.115303     0.0413578
  0.182601   -0.118858     0.194361    -0.182601      -0.0945965    0.0705856
  0.187318    0.076373    -0.137532    -0.187318       0.0716388    0.0595055
  0.319698    0.0054267   -0.00527205  -0.319698       0.0232456    0.0184745
 -0.0608038  -0.0319392    0.00885012   0.0608038  …  -0.00218701  -0.0144408
  0.231392    0.14866     -0.168531    -0.231392      -0.0897261   -0.0869261
  0.324309   -0.00623335   0.00968003  -0.324309       0.0194      -0.0301037
  0.271944   -0.124611     0.106429    -0.271944       0.0813821   -0.0779456
  0.11021     0.0672919   -0.0258803   -0.11021       -0.0317626   -0.0312472
  0.232806   -0.122968     0.0680601   -0.232806   …   0.0807878   -0.0756252
  0.0629086  -0.0152029    0.0705071   -0.0629085     -0.0307569    0.0138409
  0.0216609   0.426873    -0.446537    -0

In [8]:
using LinearAlgebra

gamma::Float64 = 1.2

for i::Int = 1:3
    cs::Matrix{Float64}, residuals::Vector{Float64}, _ = GreedyRegression.greedy_regression(G)
    k::Int = ReportModel.report_identified_model(cs, residuals, scales, labels, gamma)

    col_norms::Vector{Float64} = [norm(G[:, j] .* cs[j, k]) for j = 1:size(G, 2)]
    kill::Int = argmax(col_norms)
    G = G[:, setdiff(1:end, kill)]
    labels = labels[setdiff(1:end, kill)]
    scales = scales[setdiff(1:end, kill)]
end

[-0.9999999574817606, -1.0]
["∂u/∂x", "∂v/∂y"]


∂u/∂x + ∂v/∂y = 0


[1.0, 0.9982664110842804, 0.9990281709202437, -0.010063533719629892, -0.010076071644239326, 0.9987170027029793]
["∂v/∂t", "∂uv/∂x", "∂vv/∂y", "∂^2v/∂x^2", "∂^2v/∂y^2", "∂P/∂y"]


∂v/∂t + ∂uv/∂x + ∂vv/∂y - 0.01∂^2v/∂x^2 - 0.01∂^2v/∂y^2 + ∂P/∂y = 0


[1.0, 0.9982840493217502, 0.9989900141641198, -0.01008959656186484, -0.010034733048639883, 0.9985433628007491]
["∂u/∂t", "∂uu/∂x", "∂uv/∂y", "∂^2u/∂x^2", "∂^2u/∂y^2", "∂P/∂x"]


∂u/∂t + ∂uu/∂x + ∂uv/∂y - 0.01∂^2u/∂x^2 - 0.01∂^2u/∂y^2 + ∂P/∂x = 0


